In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter



pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
plt.rcParams['figure.figsize'] = (10,6)
sns.set(style="whitegrid")

Import File


In [ ]:
df = pd.read_csv("/kaggle/input/google-play-store-apps/googleplaystore.csv")
df.head()
df.info()


In [ ]:
print("Rows, cols:", df.shape)
df.describe(include='all').T


In [ ]:
# Convert rating to numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')


In [ ]:
df['installs_clean'] = (df['installs']
                        .astype(str)
                        .str.replace(r'[+,]', '', regex=True)
                        .str.replace('Free', '')  # if any noise
                        .replace('', np.nan)
                       )
df['installs_clean'] = pd.to_numeric(df['installs_clean'], errors='coerce')


In [ ]:
df['price_clean'] = (df['price']
                     .astype(str)
                     .str.replace(r'[$,]', '', regex=True)
                     .replace('Everyone', np.nan)  # if noisy
                    )
df['price_clean'] = pd.to_numeric(df['price_clean'], errors='coerce').fillna(0.0)


In [ ]:
def parse_size(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s.lower() in ['varies with device', 'varies with device', '']:
        return np.nan
    if s.endswith('M'):
        try: return float(s[:-1])
        except: return np.nan
    if s.endswith('k'):
        try: return float(s[:-1]) / 1024
        except: return np.nan
    # sometimes '1,000' or '1.5' already
    try:
        return float(s)
    except:
        return np.nan

df['size_mb'] = df['size'].apply(parse_size)


In [ ]:
df['last_updated'] = pd.to_datetime(df['last_updated'], errors='coerce')


In [ ]:
df['type'] = df['type'].str.strip().fillna('Free')
df['category'] = df['category'].str.strip()
df['content_rating'] = df['content_rating'].str.strip()
df['genres'] = df['genres'].str.strip()


In [ ]:
df['reviews_num'] = pd.to_numeric(df['reviews'], errors='coerce')


In [ ]:
# drop exact duplicates
df = df.drop_duplicates()

# optional: keep first occurrence of same App name (or aggregate)
df = df.sort_values('last_updated').drop_duplicates(subset=['app'], keep='last')


In [ ]:
missing = df.isna().mean().sort_values(ascending=False)
print(missing[missing>0])


5 — Feature engineering (metrics of success)

Create metrics to measure app success:

success_score — simple composite (weighted) metric combining normalized rating, installs, and reviews.

is_popular — installs > 1,000,000 (thresholds can be tuned).

days_since_update — recency metric.

In [ ]:
# normalized components (0..1)
def minmax(s):
    s = s.fillna(0)
    if s.max() == s.min():
        return pd.Series(0, index=s.index)
    return (s - s.min()) / (s.max() - s.min())

df['rating_norm'] = minmax(df['rating'])
df['installs_norm'] = minmax(df['installs_clean'])
df['reviews_norm'] = minmax(df['reviews_num'])

# weights: e.g., rating 0.5, installs 0.3, reviews 0.2
df['success_score'] = df['rating_norm']*0.5 + df['installs_norm']*0.3 + df['reviews_norm']*0.2

df['is_popular_1m'] = df['installs_clean'] >= 1_000_000
df['days_since_update'] = (pd.Timestamp.today() - df['last_updated']).dt.days


**Basic-level questions (code + notes)**# . * 

**1. What is the average rating of apps in the dataset?**

In [ ]:
df['rating'].mean(skipna=True)


**2. How many unique categories of apps are there?**

In [ ]:
df['category'].nunique()
df['category'].unique()


**3. What is the distribution of app sizes?**

In [ ]:
plt.hist(df['size_mb'].dropna(), bins=50)
plt.xlabel('Size (MB)')
plt.ylabel('Count')
plt.title('Distribution of app sizes (MB)')
plt.show()


Also consider log-scale for heavy-tailed distributions:


In [ ]:
plt.hist(np.log1p(df['size_mb'].dropna()), bins=50)


**4. How many free vs paid apps are there?**

In [ ]:
df['type'].value_counts()
sns.countplot(data=df, x='type')
plt.title('Free vs Paid apps')
plt.show()


**5. What is the most common content rating for apps?**

In [ ]:
df['content_rating'].mode()
df['content_rating'].value_counts()


**6. What are the top 5 most installed apps?**

In [ ]:
top5 = df.sort_values('installs_clean', ascending=False).head(5)[['app','installs_clean','rating','reviews_num','category']]
top5


**7.Apps with rating >= 4.0:**

In [ ]:
(df['rating'] >= 4.0).sum()


**8.Average number of reviews for free vs paid**

In [ ]:
df.groupby('type')['reviews_num'].mean().sort_values(ascending=False)


**9.Average app size per category:**

In [ ]:
df.groupby('category')['size_mb'].mean().sort_values(ascending=False).head(20)


**10.Apps last updated in 2018:**

In [ ]:
df[df['last_updated'].dt.year == 2018].shape[0]


**Medium-Level Questions:**

**1. What is the correlation between the number of installs and the app rating?**

In [ ]:

sub = df[['installs_clean','rating']].dropna()
sub['installs_log'] = np.log1p(sub['installs_clean'])
sub.corr()['rating'].sort_values()
sns.scatterplot(x='installs_log', y='rating', data=sub, alpha=0.5)
plt.title('Log(Installs) vs Rating')
plt.show()


Compute Pearson correlation on log(installs):


In [ ]:
sub['installs_log'].corr(sub['rating'])


**2. Which app categories have the highest average rating?**

In [ ]:
cat_rating = df.groupby('category')['rating'].agg(['mean','count']).sort_values('mean', ascending=False)
cat_rating.head(15)


**3. How does the price of an app affect its average rating?**

In [ ]:
paid = df[df['type']=='Paid'].copy()
plt.scatter(paid['price_clean'], paid['rating'])
plt.xscale('log')   # prices may be skewed
plt.xlabel('Price (USD)')
plt.ylabel('Rating')
plt.title('Price vs Rating (Paid apps)')
plt.show()


**4.Ratings distribution across content rating**

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x='content_rating', y='rating', data=df)
plt.title('Rating distribution by Content Rating')
plt.show()


**5.Genres with most apps >1M installs:**

In [ ]:
g = df[df['installs_clean'] > 1_000_000]
g.groupby('genres')['app'].count().sort_values(ascending=False).head(20)


**6. How frequently do apps get updated? Calculate the average time between updates.**

In [ ]:
df['days_since_update'].describe()
sns.histplot(df['days_since_update'].dropna(), bins=50)
plt.title('Days since last update')


**7.Impact of size on installs:**

In [ ]:
s = df[['size_mb','installs_clean']].dropna()
s['size_log'] = np.log1p(s['size_mb'])
s['installs_log'] = np.log1p(s['installs_clean'])
sns.scatterplot(x='size_log', y='installs_log', data=s, alpha=0.5)
plt.title('Log(Size) vs Log(Installs)')
plt.show()
s['size_log'].corr(s['installs_log'])


**8.Highest number of reviews & their ratings:**

In [ ]:
df.sort_values('reviews_num', ascending=False).head(10)[['app','reviews_num','rating','installs_clean']]


**9. How does the content rating distribution differ between free and paid apps?**

In [ ]:
pd.crosstab(df['content_rating'], df['type'], normalize='columns')  # or rows


**10.Top 5 categories with most installs:**

In [ ]:
cat_installs = df.groupby('category')['installs_clean'].sum().sort_values(ascending=False).head(10)
cat_installs.plot(kind='bar')
plt.title('Total installs by category (top 10)')
plt.show()


**Advanced-Level Questions:****

**1. What are the top 10 apps with the highest ratings, and how do their number of reviews and installs compare?**

In [ ]:
top_rated = df.sort_values(['rating','reviews_num'], ascending=[False,False]).head(20)[['app','rating','reviews_num','installs_clean','category']]


**2. Analyze the trend of app updates over time. Are there any noticeable patterns or seasonal trends?**

In [ ]:
df['year'] = df['last_updated'].dt.year
df['month'] = df['last_updated'].dt.to_period('M')
df['month'].value_counts().sort_index().plot()
plt.title('Apps: last updated month distribution')
plt.show()


**3. How does the average rating of apps change with the number of installs? Create a binned analysis.**

In [ ]:
bins = [0, 1_000, 10_000, 100_000, 1_000_000, 10_000_000, df['installs_clean'].max()]
labels = ['<1k', '1k-10k', '10k-100k', '100k-1M', '1M-10M', '10M+']

df['inst_bin'] = pd.cut(df['installs_clean'], bins=bins, labels=labels, include_lowest=True)


rating_by_installs = df.groupby('inst_bin', observed=False)['rating'].mean()

print(rating_by_installs)






In [ ]:
sns.barplot(x=rating_by_installs.index, y=rating_by_installs.values, palette='viridis')
plt.xlabel('Install Range')
plt.ylabel('Average Rating')
plt.title('Average Rating vs. Install Range')
plt.show()

**4. Perform sentiment analysis on app reviews (if review text is available) to determine the common themes in high and low-rated apps.**

In [ ]:
# Load both datasets
apps = pd.read_csv("/kaggle/input/google-play-store-apps/googleplaystore.csv")
reviews = pd.read_csv("/kaggle/input/google-play-store-apps/googleplaystore_user_reviews.csv")

print(apps.shape, reviews.shape)
reviews.head()


In [ ]:
reviews.columns = reviews.columns.str.lower().str.replace(' ', '_')
reviews = reviews.dropna(subset=['translated_review'])

# Quick look at sentiment distribution
reviews['sentiment'].value_counts()


In [ ]:
# Clean column names: strip spaces, lowercase
apps.columns = apps.columns.str.strip().str.lower()
reviews.columns = reviews.columns.str.strip().str.lower()

# Check again
print(apps.columns)
print(reviews.columns)


In [ ]:
print("Apps columns:", apps.columns.tolist())
print("Reviews columns:", reviews.columns.tolist())


In [ ]:
df_merged = pd.merge(reviews, apps, on='app', how='inner')


In [ ]:
df_merged['rating'] = pd.to_numeric(df_merged['rating'], errors='coerce')


In [ ]:
print(df_merged.shape)
df_merged.head(2)


In [ ]:
# Count sentiment labels
sentiment_counts = df_merged['sentiment'].value_counts()
print(sentiment_counts)

# Average sentiment polarity per app
sentiment_by_app = df_merged.groupby('app')['sentiment_polarity'].mean().reset_index()

# Merge with main app info
df_sent = apps.merge(sentiment_by_app, on='app', how='left')


In [ ]:
sns.scatterplot(x='sentiment_polarity', y='rating', data=df_sent, alpha=0.5)
plt.title('App Rating vs. Average Review Sentiment')
plt.xlabel('Average Sentiment Polarity (-1 to 1)')
plt.ylabel('App Rating')
plt.show()

# Correlation
df_sent[['rating','sentiment_polarity']].corr()



In [ ]:
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop = set(stopwords.words('english'))
# Separate positive and negative reviews
pos_reviews = ' '.join(df_merged[df_merged['sentiment'] == 'Positive']['translated_review'].dropna().tolist())
neg_reviews = ' '.join(df_merged[df_merged['sentiment'] == 'Negative']['translated_review'].dropna().tolist())


# Word clouds
wordcloud_pos = WordCloud(width=800, height=400, background_color='white', stopwords=stop).generate(pos_reviews)
wordcloud_neg = WordCloud(width=800, height=400, background_color='black', stopwords=stop).generate(neg_reviews)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.axis('off')
plt.title('🌟 Positive Review Themes')
plt.show()

plt.figure(figsize=(10,5))
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.axis('off')
plt.title('⚠️ Negative Review Themes')
plt.show()

In [ ]:
cat_sentiment = df_merged.groupby('category')['sentiment_polarity'].mean().sort_values(ascending=False)
cat_sentiment.plot(kind='bar', figsize=(12,5))
plt.title('Average Sentiment Polarity by App Category')
plt.ylabel('Average Polarity')
plt.show()


**5. What is the relationship between app genre and user ratings? Are certain genres consistently rated higher or lower?**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Clean column names
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

# Drop missing or invalid ratings
df = df[df['rating'].notna()]

# Extract the primary genre (some have multiple, e.g., "Action;Adventure")
df['primary_genre'] = df['genres'].apply(lambda x: x.split(';')[0] if isinstance(x, str) else x)


In [ ]:
genre_rating = df.groupby('primary_genre')['rating'].agg(['mean', 'median', 'count']).reset_index()
genre_rating = genre_rating.sort_values(by='mean', ascending=False)

print(genre_rating.head(10))


In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x='primary_genre', y='mean', data=genre_rating, palette='viridis')
plt.xticks(rotation=75, ha='right')
plt.title('Average User Rating by App Genre')
plt.ylabel('Average Rating')
plt.xlabel('App Genre')
plt.show()


In [ ]:
top_genres = genre_rating[genre_rating['count'] > 50].head(5)
bottom_genres = genre_rating[genre_rating['count'] > 50].tail(5)

print("Top Rated Genres:\n", top_genres[['primary_genre','mean']])
print("\nLowest Rated Genres:\n", bottom_genres[['primary_genre','mean']])


*The analysis reveals that user satisfaction varies significantly by app genre. Puzzle, Art & Design, and Education apps achieve the highest average ratings (above 4.3), indicating strong user engagement and perceived value. In contrast, Dating, Tools, and Video Players & Editors apps show relatively lower ratings (around 4.0), suggesting usability or performance challenges. These insights highlight that genres emphasizing creativity and learning attract more positive feedback, while functional or utility-based apps may require experience enhancements to boost user satisfaction.*